# Numba vs. Cython: Take 2

*This notebook first appeared as a*
[*post*](http://jakevdp.github.io/blog/2012/08/24/numba-vs-cython/)
*by Jake Vanderplas on the blog*
[*Pythonic Perambulations*](http://jakevdp.github.io)

In [1]:
import numpy as np
X = np.random.random((1000, 3))

## Pure Python Function

A loop-based solution avoids the overhead associated with temporary arrays,
and can be written like this:

In [2]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D
%timeit -n 1 -r 1 pairwise_python(X)

1 loop, best of 1: 4.77 s per loop


## Numpy Function With Broadcasting

We'll start with a typical numpy broadcasting approach to this problem.  Numpy
broadcasting is an abstraction that allows loops over array indices to be
executed in compiled C.  For many applications, this is extremely fast and efficient.
Unfortunately, there is a problem with broadcasting approaches that comes up here:
it ends up allocating hidden temporary arrays which can eat up memory and cause
computational overhead.  Nevertheless, it's a good comparison to have.  The function
looks like this:

In [3]:
def pairwise_numpy(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))
%timeit -n 1 -r 1 pairwise_numpy(X)

1 loop, best of 1: 89.9 ms per loop


## Numba Wrapper

[Numba](http://numba.pydata.org/) is an LLVM compiler for python code, which
allows code written in Python to be converted to highly efficient compiled code
in real-time.  Due to its dependencies, compiling it can be a challenge.  To experiment
with Numba, I recommend using a local installation of [Anaconda](https://store.continuum.io/),
the free cross-platform Python distribution which includes Numba and all its prerequisites
within a single easy-to-install package.

Numba is extremely simple to use.  We just wrap our python function with ``autojit`` (JIT stands
for "just in time" compilation) to automatically create an efficient, compiled version of the function:

In [4]:
from numba import double
from numba.decorators import jit
from math import sqrt

In [5]:
@jit
def pairwise_numba(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return D

# call once to compile?
_ = pairwise_numba(X)

In [6]:
%timeit -n 1 pairwise_numba(X)

1 loop, best of 3: 5.04 ms per loop


Adding this simple expression speeds up our execution by over a factor of over 1400!
For those keeping track, this is about 50% faster than the version of Numba that
I tested last August on the same machine.

## Optimized Cython Function

[Cython](http://cython.org) is another package which is built to convert Python-like statemets
into compiled code. The language is actually a superset of Python which acts as a sort of
hybrid between Python and C.  By adding type annotations to Python code and running
it through the Cython interpreter, we obtain fast compiled code.  Here is a
highly-optimized Cython version of the pairwise distance function, which we compile
using IPython's Cython magic:

In [7]:
%load_ext Cython

In [8]:
%%cython
import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((M, M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return np.asarray(D)

In [9]:
%timeit -n 1 pairwise_cython(X)

1 loop, best of 3: 5.78 ms per loop


## Scipy Pairwise Distances

Because pairwise distances are such a commonly used application in scientific
computing, both Scipy and scikit-learn have optimized routines to compute them.
The Scipy version is a Python wrapper of C code, and can be called as follows:

In [141]:
from scipy.spatial.distance import cdist

In [148]:
%timeit -n 1 cdist(X, X)

1 loop, best of 3: 5.05 ms per loop


## Scikit-learn Pairwise Distances

Scikit-learn contains the ``euclidean_distances`` function, works on sparse
matrices as well as numpy arrays, and is implemented in Cython:

In [149]:
from sklearn.metrics import euclidean_distances

In [150]:
%timeit -n 2 euclidean_distances(X, X)

2 loops, best of 3: 8.85 ms per loop


# MDDist

In [156]:
from MDAnalysis.analysis.distances import distance_array

In [158]:
X32 = X.astype(np.float32)

In [163]:
%timeit -n 1 distance_array(X32, X32) 

1 loop, best of 3: 5 ms per loop
